In [110]:
import pandas as pd

from bokeh.io import show, output_notebook
from bokeh.layouts import widgetbox, row
from bokeh.models.widgets import RadioGroup
from bokeh.models import CustomJS ,ColumnDataSource
from bokeh.plotting import figure


In [111]:
df = pd.read_excel('TPbars.xlsx')
source = ColumnDataSource(df)
df

,Scenario,Retail price,Social cost,Water cost
0,Normal rose,10,3,5
1,Water saving rose,9,3,2
2,Living wage rose,11,1,5
3,Water saving & living wage rose,10,1,2


In [112]:
scenarios=list(df["Scenario"])
scenarios

['Normal rose',
 'Water saving rose',
 'Living wage rose',
 'Water saving & living wage rose']

In [113]:
retail=list(df["Retail price"])
retail


[10, 9, 11, 10]

In [114]:
n = 0

In [115]:
plot = figure(plot_width=600, plot_height=400, x_range=scenarios, title="Price of a Kenyan rose", tools="", logo="grey")
a= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Retail price", 
    color="MidnightBlue", 
    legend="Market price",
    visible=True,
    source=source
    )


In [116]:
widgets = RadioGroup(labels=["Scenario 1", "Scenario 2","Scenario 3","Scenario 4"], active=0)


In [125]:
def callback(attr, old, new):
    a.visible=False


widgets.on_change("active", callback)

In [126]:
output_notebook()

Loading BokehJS ...

In [127]:
layout=(row(widgetbox(widgets),plot))
show(layout)

In [ ]:
'''
plot.vbar(
    x=[scenarios], 
    width=0.5, 
    top=df["Retail price"]+df["Social cost"], 
    color = "FireBrick", 
    bottom = df["Retail price"], 
    legend="Social cost")
plot.vbar(
    x=[scenarios], 
    width=0.5, 
    top=df["Retail price"]+df["Social cost"]+df["Water cost"], 
    color = "SeaGreen", 
    bottom = df["Retail price"]+df["Social cost"], 
    legend="Water cost"
    '''
    